In [51]:
import numpy as np


def Usolve(U, y, unit_diag=False):
    """Backward solve an upper triangular system Ux = y for x
    Parameters: 
      U: the matrix, must be square, upper triangular, with nonzeros on the diagonal
      y: the right-hand side vector
      unit_diag: if True, assume the diagonal is all ones
    Output:
      x: the solution vector to U @ x == y
    """
    # Check the input
    m, n = U.shape
    assert m == n, "matrix must be square"
    assert np.all(np.triu(U) == U), "matrix U must be upper triangular"
    if unit_diag:
        assert np.all(np.diag(U) == 1), "matrix U must have ones on the diagonal"
    yn, = y.shape
    assert yn == n, "rhs vector must be same size as U"
    
    # Make a copy of y that we will transform into the solution
    x = y.astype(np.float64).copy()
    
    # Back solve
    for col in reversed(range(n)):
        if not unit_diag:
            x[col] /= U[col, col]
        x[:col] -= x[col] * U[:col, col]
        
    return x

def Lsolve(L, b, unit_diag=False):
    """Forward solve a unit lower triangular system Ly = b for y
    Parameters: 
      L: the matrix, must be square, lower triangular, with ones on the diagonal
      b: the right-hand side vector
      unit_diag: if True, assume the diagonal is all ones
    Output:
      y: the solution vector to L @ y == b
    """
    
    # Check the input
    m, n = L.shape
    assert m == n, "matrix L must be square"
    assert np.all(np.tril(L) == L), "matrix L must be lower triangular"
    if unit_diag:
        assert np.all(np.diag(L) == 1), "matrix L must have ones on the diagonal"
    
    # Make a copy of b that we will transform into the solution
    y = b.astype(np.float64).copy()
    
    # Forward solve
    for col in range(n):
        if not unit_diag:
            y[col] /= L[col, col]
        y[col+1:] -= y[col] * L[col+1:, col]
        
    return y


# Permutation Matrices

In [12]:
# Example:

A = np.array([[0, 1, 3, 1], 
              [1, 6, 2, 0],
              [0, 0, 0, 2],
              [0, 0, 6, 1]])
b = np.array([5,4,3,2])

In [13]:
# How can we permute A to make it upper trianglular?

In [14]:
A_p1 = A[[1,0,3,2]]
A_p1


array([[1, 6, 2, 0],
       [0, 1, 3, 1],
       [0, 0, 6, 1],
       [0, 0, 0, 2]])

In [15]:
P = np.eye(4)[[1,0,3,2]]
print(A_p1)
print(P @ A) 
#P

[[1 6 2 0]
 [0 1 3 1]
 [0 0 6 1]
 [0 0 0 2]]
[[1. 6. 2. 0.]
 [0. 1. 3. 1.]
 [0. 0. 6. 1.]
 [0. 0. 0. 2.]]


In [17]:
ans1 = np.linalg.solve(A,b)

# why doesn't the following line work:
ans2 = Usolve(A_p1, b)



print(ans1)
print(ans2)

[-15.66666667   3.25         0.08333333   1.5       ]
[-7.66666667  2.          0.33333333  1.        ]


In [18]:
ans3 = Usolve(A_p1, P @b)
print(ans1)
print(ans3)

[-15.66666667   3.25         0.08333333   1.5       ]
[-15.66666667   3.25         0.08333333   1.5       ]


In [19]:
#What is a permutation matrix times another permutation matrix?

# Symmetric Positive Definite Matrices

Symmetric if a_ij = a_ji for all i, j 
          if A= A.T

Q Positive Definite if:
           - all eigenvalues are >0
           - LU factorization without pivoting succeeeds and all pivots are > 0
           - For every nonzero vector x: xQx.T >0

# Decomposition

Let's say we know `A = D*E`, how can we use D and E to solve for Ax=b?

We have `D*E*x = b`


Let `E*x=z`

First, solve for z in `D*z=b` 

Then solve for `E*x=z`

|  Name | D | E  | 
|---|---|---|
|LU   | Lower Unit Trianlge  |Upper Triangle   | 
|QR   |  Orthogonal | Upper Triangular  |   
|Cholesky   |Lower Triangular  | D.T  |   

Notes:
*Orthogonal holds the following equivalent statements:

-Columns and rows are all unit vectors

-inv(Q)=Q.T

as a result, columns of Q are mutually perpendicular


Cholesky factorization only works on Positive Definite Matrices



In [53]:
## Cholesky Decomposition

A = np.array([[1, 3, 4], [3, 10, 14], [4, 14, 21]])
b = np.array([4,1,6])
A

array([[ 1,  3,  4],
       [ 3, 10, 14],
       [ 4, 14, 21]])

In [46]:
#print(A)
#print(np.linalg.eig(A)[0])
L = np.linalg.cholesky(A)
print(L)

[[1. 0. 0.]
 [3. 1. 0.]
 [4. 2. 1.]]


In [47]:
#print(L )
print(L @ L.T)

[[ 1.  3.  4.]
 [ 3. 10. 14.]
 [ 4. 14. 21.]]


In [54]:
# No we solve for Lz = b

y = Lsolve(L, b)
ans = Usolve(L.T, y)
print(ans)


print(np.linalg.solve(A, b))

[ 61. -35.  12.]
[ 61. -35.  12.]
